In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/events.csv')
df.sample(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
637942,2021-01-19 08:31:44 UTC,view,367545,2144415925011480748,stationery.cartrige,nv-print,14.08,1515915625594779950,Yw0ya2Ke1F
334840,2020-11-22 13:25:49 UTC,view,3790736,2144415923107266682,computers.peripherals.printer,canon,163.52,1515915625533720850,BIVVMPrjER
264073,2020-11-12 08:01:43 UTC,view,4100720,2144415927972659448,computers.notebook,hama,63.02,1515915625538451325,8vdSFxnjvi
824603,2021-02-18 02:03:42 UTC,view,698803,2144415942350733951,NaN,hammer,150.78,1515915625606700663,rjFhTi1BKI
703252,2021-01-29 11:52:46 UTC,view,1400096,2144415927251239140,NaN,pitatel,205.08,1515915625598358081,Xd1eayKrGO


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885129 entries, 0 to 885128
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     885129 non-null  object 
 1   event_type     885129 non-null  object 
 2   product_id     885129 non-null  int64  
 3   category_id    885129 non-null  int64  
 4   category_code  648910 non-null  object 
 5   brand          672765 non-null  object 
 6   price          885129 non-null  float64
 7   user_id        885129 non-null  int64  
 8   user_session   884964 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 60.8+ MB


In [ ]:
df.isnull().sum()

,0
event_time,0
event_type,0
product_id,0
category_id,0
category_code,236219
brand,212364
price,0
user_id,0
user_session,165


# 1차 전처리

In [ ]:
# user_session&brand null인 행 165개 제거
df = df.dropna(subset=['user_session'])

df.isnull().sum()

,0
event_time,0
event_type,0
product_id,0
category_id,0
category_code,236172
brand,212326
price,0
user_id,0
user_session,0


In [ ]:
# 사용자 로그 데이터 분석을 위한 데이터셋
    # category_code, brand 컬럼 사용 X
log_df = df[['event_time','event_type','product_id','category_id', 'price', 'user_id', 'user_session']]
log_df.sample(5)

,event_time,event_type,product_id,category_id,price,user_id,user_session
111997,2020-10-19 00:50:50 UTC,view,1355092,2144415934608048556,23.44,1515915625527590924,9ylbIDtddU
774778,2021-02-09 20:02:17 UTC,view,622727,2144415927158964449,7.41,1515915625591534520,bQA4DOsDYC
424647,2020-12-11 05:14:57 UTC,view,1529023,2144415924021624977,16.19,1515915625559849358,CekjGGX4AG
607893,2021-01-14 12:31:54 UTC,view,4078973,2144415935329468864,55.87,1515915625555548801,Fkyyi2xI3f
489071,2020-12-23 06:08:47 UTC,view,1503549,2144415942904382094,25.08,1515915625567359568,CTCYIupgkr


# DAU


## DAU 추이

In [ ]:
# 데이터 타입 변경
log_df['event_time'] = pd.to_datetime(log_df['event_time'], format= '%Y-%m-%d %H:%M:%S UTC')

<ipython-input-60-44533b88f417>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
log_df['date_ymd'] = log_df['event_time'].dt.date # 시간 제거하고 날짜만
log_df['date_ymd'] = pd.to_datetime(log_df['date_ymd'], format= '%Y-%m-%d') #데이터 타입 변경
log_df.head(3)

<ipython-input-61-0dd18d4321c3>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-61-0dd18d4321c3>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,event_time,event_type,product_id,category_id,price,user_id,user_session,date_ymd
0,2020-09-24 11:57:06,view,1996170,2144415922528452715,31.90,1515915625519388267,LJuJVLEjPT,2020-09-24
1,2020-09-24 11:57:26,view,139905,2144415926932472027,17.16,1515915625519380411,tdicluNnRY,2020-09-24
2,2020-09-24 11:57:27,view,215454,2144415927158964449,9.81,1515915625513238515,4TMArHtXQy,2020-09-24


In [ ]:
# 월별 데이터프레임
oct_df = log_df[log_df['date_ymd'].dt.month == 10]
nov_df = log_df[log_df['date_ymd'].dt.month == 11]
dec_df = log_df[log_df['date_ymd'].dt.month == 12]
jan_df = log_df[log_df['date_ymd'].dt.month == 1]
feb_df = log_df[log_df['date_ymd'].dt.month == 2]

In [ ]:
# 일자별 user_id 고유갯수를 구해줌
oct_dau = oct_df.groupby('date_ymd')['user_id'].nunique().reset_index().rename({'user_id': 'dau'}, axis = 1)
oct_dau

,date_ymd,dau
0,2020-10-01,2699
1,2020-10-02,2558
2,2020-10-03,2162
3,2020-10-04,2398
4,2020-10-05,2832
5,2020-10-06,2676
6,2020-10-07,2560
7,2020-10-08,2660
8,2020-10-09,2414
9,2020-10-10,2114


In [ ]:
import plotly.express as px
fig = px.line(data_frame= oct_dau, x = 'date_ymd', y = 'dau', title= '10월 DAU 추이')
fig.show()

- 10월 한달 간 DAU 수치가 크게 떨어진 포인트는 4곳
  - 10/3, 10/10, 10/17, 10/24
  - 모두 동일한 요일임을 알 수 있음
  - 주기성이 있는지 분석해보기로 결정

In [ ]:
# 요일별로 DAU 추이에 차이가 있는지
oct_dau['day_of_week'] = oct_dau['date_ymd'].dt.day_name()
oct_dau['day_of_week1'] = oct_dau['date_ymd'].dt.day_of_week # 요일을 숫자로 표현
oct_dau.head()

,date_ymd,dau,day_of_week,day_of_week1
0,2020-10-01,2699,Thursday,3
1,2020-10-02,2558,Friday,4
2,2020-10-03,2162,Saturday,5
3,2020-10-04,2398,Sunday,6
4,2020-10-05,2832,Monday,0


In [ ]:
# 요일별 평균
avg_dau_by_dow = oct_dau.groupby(['day_of_week', 'day_of_week1'])['dau'].mean().reset_index()
avg_dau_by_dow.sort_values('day_of_week1', inplace= True)
avg_dau_by_dow

,day_of_week,day_of_week1,dau
1,Monday,0,3221.00
5,Tuesday,1,3133.50
6,Wednesday,2,3085.25
4,Thursday,3,3104.60
0,Friday,4,3006.20
2,Saturday,5,2592.40
3,Sunday,6,2747.25


In [ ]:
fig = px.bar(data_frame= avg_dau_by_dow, x = 'day_of_week', y = 'dau', title = '요일별 DAU 평균', height= 400, width= 600)
fig.show()

## 사이트 체류 시간

In [ ]:
oct_df.sample(3)

,event_time,event_type,product_id,category_id,price,user_id,user_session,date_ymd
68601,2020-10-10 02:56:18,view,1747855,2144415942703055498,140.63,1515915625524205235,TFapx4heZE,2020-10-10
180121,2020-10-30 09:12:56,view,788861,2144415926185885895,59.44,1515915625532193007,qDWGXkmX8V,2020-10-30
119165,2020-10-20 05:47:59,view,1529071,2144415922587172973,56.51,1515915625528166466,XmY4ZmF8Gj,2020-10-20


In [ ]:
oct_df.query('user_session == "feye4Xb5rR"').sort_values('event_time')

,event_time,event_type,product_id,category_id,price,user_id,user_session,date_ymd
116033,2020-10-19 14:33:39,view,846904,2144415922016747613,149.05,1515915625527955680,feye4Xb5rR,2020-10-19
116038,2020-10-19 14:34:13,cart,846904,2144415922016747613,149.05,1515915625527955680,feye4Xb5rR,2020-10-19
116044,2020-10-19 14:34:34,purchase,846904,2144415922016747613,149.05,1515915625527955680,feye4Xb5rR,2020-10-19


In [ ]:
# 유저별 사이트 체류시간 평균
oct_duration = oct_df.groupby('user_session')[['event_time']].agg(['max','min']).reset_index()
oct_duration.sample(5)

user_session          event_time                    
                                   max                 min
33007   L9yiVlBnkY 2020-10-07 07:50:23 2020-10-07 07:50:23
45538   TSO3feU5c4 2020-10-01 20:47:43 2020-10-01 20:47:43
81181   qb044fj9xb 2020-10-29 09:39:21 2020-10-29 09:39:21
36236   NJnv0nAEy0 2020-10-03 20:25:12 2020-10-03 20:25:12
73282   lRSuXbFclN 2020-10-17 08:32:51 2020-10-17 08:32:40

In [ ]:
oct_duration['duration'] = oct_duration['event_time']['max'] - oct_duration['event_time']['min']
oct_duration.sample(10)

user_session          event_time                            duration
                                   max                 min                
41906   R2oQK1QQID 2020-10-20 10:58:30 2020-10-20 10:58:30 0 days 00:00:00
16237   AD9LpZf5xd 2020-10-14 17:57:39 2020-10-14 17:47:29 0 days 00:10:10
34314   M1rX7puLBq 2020-10-06 22:02:31 2020-10-06 22:02:31 0 days 00:00:00
56107   aP8FKMr4bA 2020-10-17 16:09:59 2020-10-17 16:09:59 0 days 00:00:00
80958   qS3nPgDmal 2020-10-12 11:38:26 2020-10-12 11:38:26 0 days 00:00:00
71624   kNW238xuXW 2020-10-16 22:49:32 2020-10-16 22:49:32 0 days 00:00:00
69584   j2Bhu8wuWO 2020-10-12 07:42:45 2020-10-12 07:07:37 0 days 00:35:08
51226   XDPFGAUVVX 2020-10-16 06:30:33 2020-10-16 06:30:33 0 days 00:00:00
41562   QonxI7i8Z8 2020-10-16 09:47:24 2020-10-16 09:47:24 0 days 00:00:00
29181   IYuuYDb1gj 2020-10-18 15:59:08 2020-10-18 15:38:00 0 days 00:21:08

In [ ]:
oct_duration.columns = ['user_session', 'max', 'min', 'duration']

In [ ]:
# 체류시간 평균 구하기
oct_duration['duration'].mean()

Timedelta('0 days 02:37:11.304356452')

In [ ]:
oct_session_pivot = pd.pivot_table(data = oct_df, index= 'user_session', columns= 'event_type', values='event_time', aggfunc= 'count').reset_index().fillna(0)
oct_session_pivot.head(10)

event_type,user_session,cart,purchase,view
0,003QqC0jk0,0.0,0.0,1.0
1,0041IWrUEl,0.0,0.0,1.0
2,00ATIKSi49,0.0,0.0,1.0
3,00B4xBIIZJ,1.0,0.0,6.0
4,00Cb9Z13da,0.0,0.0,2.0
5,00CbJ5oeHx,0.0,0.0,1.0
6,00CcHZJiqz,0.0,0.0,1.0
7,00JP8MmKsg,0.0,0.0,1.0
8,00MA8dpitN,0.0,0.0,1.0
9,00QVxHRW1L,0.0,0.0,1.0


In [ ]:
# 카트 세션과, 구매 세션 구분
cart_session = list(oct_session_pivot.query('cart > 0')['user_session'])
purchase_session = list(oct_session_pivot.query('purchase > 0')['user_session'])

In [ ]:
view_sesseion_avg_duration = oct_duration.query('user_session not in @cart_session and user_session not in @purchase_session')['duration'].mean()
cart_sesstion_avg_duration = oct_duration.query('user_session in @cart_session')['duration'].mean()
purchase_sesstion_avg_duration = oct_duration.query('user_session in @purchase_session')['duration'].mean()

print(f'조회만 한 유저의 평균 체류시간, {view_sesseion_avg_duration}')
print(f'카트에 담은 유저의 평균 체류시간, {cart_sesstion_avg_duration}')
print(f'구매까지 한 유저의 평균 체류시간, {purchase_sesstion_avg_duration}')

조회만 한 유저의 평균 체류시간, 0 days 02:09:44.840531636
카트에 담은 유저의 평균 체류시간, 0 days 08:33:44.898582493
구매까지 한 유저의 평균 체류시간, 0 days 09:06:47.902150277


In [ ]:
oct_session_pivot

event_type,user_session,cart,purchase,view
0,003QqC0jk0,0.0,0.0,1.0
1,0041IWrUEl,0.0,0.0,1.0
2,00ATIKSi49,0.0,0.0,1.0
3,00B4xBIIZJ,1.0,0.0,6.0
4,00Cb9Z13da,0.0,0.0,2.0
...,...,...,...,...
95761,zzgv26JbgZ,0.0,0.0,1.0
95762,zzlkMPGDur,0.0,0.0,1.0
95763,zznONIlprS,0.0,0.0,1.0
95764,zztSfMn54L,0.0,0.0,1.0


In [ ]:
oct_funnel = oct_session_pivot[['view', 'cart', 'purchase']].sum().to_frame().reset_index()
oct_funnel

,event_type,0
0,view,146510.0
1,cart,8729.0
2,purchase,6276.0


In [ ]:
oct_funnel.columns = ['event_type', 'count']
oct_funnel

,event_type,count
0,view,146510.0
1,cart,8729.0
2,purchase,6276.0


In [ ]:
fig = px.funnel(data_frame= oct_funnel, x = 'event_type', y = 'count')
fig.update_traces(texttemplate= '%{value:,.0f}') # 숫자 형식 지정
fig.show()

In [ ]:
# 방문자 구매비율
purchse_ratio = list(oct_funnel['count'])[2] / list(oct_funnel['count'])[0]
print(f'방문자 구매비율: {purchse_ratio}')

방문자 구매비율: 0.04283666643915091


In [ ]:
#전환율 계산
view_to_cart_rate = list(oct_funnel['count'])[1] / list(oct_funnel['count'])[0]
view_to_purchase_rate = list(oct_funnel['count'])[2] / list(oct_funnel['count'])[1]

oct_funnel['retain_rate'] = [1,view_to_cart_rate ,  view_to_purchase_rate]

#시각화
fig = px.funnel(data_frame = oct_funnel, x = 'event_type', y = 'retain_rate')
fig.update_traces(texttemplate = '%{value:,.2%}')
fig.show()

* 전환율과 이탈률은 서로 상호보완적인 관계로, 전환율 = 1 - 이탈률로 표현할 수 있습니다.

 - 조회 → 장바구니 이탈률: 약 94%
 - 장바구니 → 구매 이탈률: 약 29%

* 조회에서 장바구니로 넘어가는 이탈율 훨씬 높음을 알 수 있음

# Event_type

In [ ]:
# '.' 구분자가 두 개 이상인 행 필터링
df_1 = df[df['category_code'].str.count('\.') >= 2]
df_1.sample(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
388874,2020-12-03 08:20:41 UTC,view,3987950,2144415926554984657,construction.tools.drill,bort,77.56,1515915625547789575,JU7SJaqGCc
695032,2021-01-28 05:38:26 UTC,cart,1423324,2144415923107266682,computers.peripherals.printer,kyocera,414.70,1515915625597897597,w3dornl45R
306251,2020-11-18 10:26:34 UTC,view,4170538,2144415922360680550,computers.peripherals.camera,cbr,14.62,1515915625540772334,NPBgjISLig
799367,2021-02-13 20:42:02 UTC,purchase,1135627,2144415923207929981,auto.accessories.player,kenwood,74.21,1515915625605187377,yhOv9Iaclh
763424,2021-02-08 07:52:51 UTC,view,659978,2144415922016747613,computers.peripherals.wifi,zyxel,32.02,1515915625602811423,WfwNomeUnM
555765,2021-01-06 07:57:20 UTC,view,4154620,2144415922427789416,computers.components.videocards,msi,656.63,1515915625573384941,MBh0iQyWXs
815114,2021-02-16 12:30:49 UTC,view,4079061,2144415922427789416,computers.components.videocards,powercolor,399.48,1515915625606145917,rR7GloLHBL
692369,2021-01-27 15:15:59 UTC,view,136700,2144415924491387038,computers.components.motherboard,asrock,77.73,1515915625597735939,zdEDKD3Jlk
416829,2020-12-09 12:14:59 UTC,view,1413340,2144415926966026460,computers.components.cpu,amd,137.63,1515915625556331837,Gw09qyFyKg
782962,2021-02-11 06:33:11 UTC,view,881920,2144415921588928594,auto.accessories.gps,navitel,80.49,1515915625604010313,KFDzSc8MSb


In [ ]:
df_1['category'] = df_1['category_code'].apply(lambda x: x.split('.')[0])
# 구분자 '.'를 기준으로 맨 뒤의 데이터를 product 열에 추가
df_1['product'] = df_1['category_code'].str.split('.').str[-1]
df_1.sample(5)

<ipython-input-103-40c4e1101bd2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-103-40c4e1101bd2>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category,product
539303,2021-01-03 08:03:16 UTC,view,1830260,2144415937812496902,construction.tools.light,philips,44.76,1515915625572166958,KzrglP4ICH,construction,light
856010,2021-02-23 16:08:37 UTC,view,872185,2144415923744800906,appliances.environment.vacuum,bosch,67.51,1515915625608844644,AtAhqkq5pB,appliances,vacuum
624521,2021-01-17 08:15:03 UTC,view,4154414,2144415922427789416,computers.components.videocards,gigabyte,202.67,1515915625593769774,0AWGkV9Mmu,computers,videocards
643329,2021-01-19 22:35:29 UTC,view,3977213,2144415922427789416,computers.components.videocards,palit,839.75,1515915625549203447,p2wXE3LwMT,computers,videocards
547304,2021-01-04 16:14:37 UTC,purchase,1756398,2144415922427789416,computers.components.videocards,asus,524.56,1515915625572772861,XBurK3w4f2,computers,videocards


In [ ]:
df_1 = df_1[['event_time','event_type','product_id','category_code','category','product','brand','price','user_id','user_session']]
df_1.sample(5)

,event_time,event_type,product_id,category_code,category,product,brand,price,user_id,user_session
512309,2020-12-27 19:01:11 UTC,view,4183863,computers.components.videocards,computers,videocards,palit,577.60,1515915625569714859,gogaxDmmhN
203545,2020-11-03 08:14:47 UTC,view,3501863,computers.components.hdd,computers,hdd,NaN,121.38,1515915625534029963,nMuhyxPpAP
869172,2021-02-26 01:01:39 UTC,cart,3538331,computers.components.videocards,computers,videocards,msi,496.54,1515915625609658303,NewzgC9Ktg
803121,2021-02-14 14:19:07 UTC,view,861964,computers.peripherals.printer,computers,printer,cactus,21.30,1515915625605444416,wruYotv0Ux
634924,2021-01-18 18:22:52 UTC,view,3791351,computers.components.cpu,computers,cpu,amd,204.29,1515915625594623235,fVl0wLvpRr


In [ ]:
# df.to_csv('/content/drive/My Drive/electronic_brand.csv', encoding='utf-8-sig', index=False)

### Product별 Brand 수

In [ ]:
brand_counts = df_1.groupby('product')['brand'].nunique().reset_index()
brand_counts.columns = ['product', 'brand_count']

# 내림차순 정렬
brand_counts = brand_counts.sort_values(by='brand_count', ascending=False)

In [ ]:
fig = px.bar(brand_counts, x='product', y='brand_count',
             title='product Brand 개수',
             labels={'brand_count': 'Brand 개수', 'product': 'Product'},
             color='brand_count',
             color_continuous_scale=px.colors.sequential.Viridis)

# 그래프 보여주기
fig.show()

In [ ]:
df_2 = pd.merge(df_1, brand_counts, on='product', how='inner')
df_2

,event_time,event_type,product_id,category_code,category,product,brand,price,user_id,user_session,brand_count
0,2020-09-24 11:57:26 UTC,view,139905,computers.components.cooler,computers,cooler,zalman,17.16,1515915625519380411,tdicluNnRY,31
1,2020-09-24 11:57:33 UTC,view,635807,computers.peripherals.printer,computers,printer,pantum,113.81,1515915625519014356,aGFYrNgC08,37
2,2020-09-24 11:57:59 UTC,view,664325,construction.tools.saw,construction,saw,carver,52.33,1515915625519388062,vnkdP81DDW,23
3,2020-09-24 11:58:24 UTC,view,716611,computers.network.router,computers,router,d-link,53.14,1515915625519388882,kVBeYDPcBw,19
4,2020-09-24 11:58:31 UTC,view,716611,computers.network.router,computers,router,d-link,53.14,1515915625519388929,F3VB9LYp39,19
...,...,...,...,...,...,...,...,...,...,...,...
448936,2021-02-28 23:51:25 UTC,view,622796,computers.components.cdrw,computers,cdrw,asus,147.38,1515915625572947504,LJ4H6CRcME,8
448937,2021-02-28 23:51:25 UTC,view,622796,computers.components.cdrw,computers,cdrw,asus,147.38,1515915625572947504,SqlXaC3Wrw,8
448938,2021-02-28 23:53:13 UTC,view,4079420,computers.components.videocards,computers,videocards,msi,449.51,1515915625611023581,zrl0oKrysT,9
448939,2021-02-28 23:58:05 UTC,view,1715907,electronics.video.tv,electronics,tv,starwind,80.03,1515915625611024014,g6WqPf50Ma,36


### Product별 event_type 세부 지표

In [ ]:
# 1. product 컬럼별로 view, cart, purchase의 총 갯수를 각 컬럼으로 만들어주기
event_counts = df_2.groupby('product')['event_type'].value_counts().unstack(fill_value=0)
event_counts.columns.name = None  # Remove the name of the columns

# 2. product별로 user_id 1명당 평균 view, cart, purchase 횟수 계산하기
user_event_counts = df_2.groupby(['product', 'user_id', 'event_type']).size().unstack(fill_value=0)

# 각 이벤트의 총합 계산
user_event_counts = user_event_counts.groupby('product').agg({
    'view': 'sum',
    'cart': 'sum',
    'purchase': 'sum'
}).reset_index()

# 평균 계산
user_count = df_2.groupby('product')['user_id'].nunique().reset_index(name='user_count')

# Merge user count with event counts
avg_counts = user_event_counts.merge(user_count, on='product')

# 각 평균을 계산하고 반올림
avg_counts['avg_view'] = (avg_counts['view'] / avg_counts['user_count']).round(3)
avg_counts['avg_cart'] = (avg_counts['cart'] / avg_counts['user_count']).round(3)
avg_counts['avg_purchase'] = (avg_counts['purchase'] / avg_counts['user_count']).round(3)

# 3. product별로 cart_conversion과 purchase_conversion 계산하기
avg_counts['cart_conversion'] = (avg_counts['cart'] / avg_counts['view'].replace(0, pd.NA)).round(3)
avg_counts['purchase_conversion'] = (avg_counts['purchase'] / avg_counts['cart'].replace(0, pd.NA)).round(3)

product_df = event_counts.merge(avg_counts[['product', 'avg_view', 'avg_cart', 'avg_purchase', 'cart_conversion', 'purchase_conversion']], on='product')

product_df = product_df.sort_values(by='view', ascending=False)
product_df = product_df[['product', 'view', 'cart', 'purchase', 'avg_view', 'avg_cart', 'avg_purchase', 'cart_conversion', 'purchase_conversion']]

product_df

,product,view,cart,purchase,avg_view,avg_cart,avg_purchase,cart_conversion,purchase_conversion
68,videocards,97141,12683,6888,3.338,0.436,0.237,0.131,0.543089
47,printer,37479,3183,2557,1.897,0.161,0.129,0.085,0.80333
0,acoustic,24776,1140,848,1.839,0.085,0.063,0.046,0.74386
37,motherboard,23221,2113,1266,2.299,0.209,0.125,0.091,0.599148
13,cpu,21314,2241,1213,2.148,0.226,0.122,0.105,0.541276
...,...,...,...,...,...,...,...,...,...
8,chair,57,0,0,1.188,0.000,0.000,0.000,<NA>
3,bath,32,1,0,1.280,0.040,0.000,0.031,0.0
4,blanket,24,2,0,1.714,0.143,0.000,0.083,0.0
51,refrigerators,15,0,0,1.250,0.000,0.000,0.000,<NA>


* 77개 product 中 구매까지 이어진 product 69개

In [ ]:
# NaN 값을 0으로 대체한 후 purchase_conversion 컬럼에 round(3) 적용
product_df['purchase_conversion'] = product_df['purchase_conversion'].fillna(0).round(3)

# purchase가 0인 행들을 삭제한 product_filtered_df 생성
product_filtered_df = product_df[product_df['purchase'] != 0].reset_index(drop=True)

# 결과 확인
product_filtered_df

<ipython-input-142-b92c7d2241c3>:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,product,view,cart,purchase,avg_view,avg_cart,avg_purchase,cart_conversion,purchase_conversion
0,videocards,97141,12683,6888,3.338,0.436,0.237,0.131,0.543
1,printer,37479,3183,2557,1.897,0.161,0.129,0.085,0.803
2,acoustic,24776,1140,848,1.839,0.085,0.063,0.046,0.744
3,motherboard,23221,2113,1266,2.299,0.209,0.125,0.091,0.599
4,cpu,21314,2241,1213,2.148,0.226,0.122,0.105,0.541
...,...,...,...,...,...,...,...,...,...
64,toster,204,3,2,1.427,0.021,0.014,0.015,0.667
65,vr,180,9,7,1.417,0.071,0.055,0.050,0.778
66,generator,176,6,2,1.323,0.045,0.015,0.034,0.333
67,heater,106,6,4,1.359,0.077,0.051,0.057,0.667


In [ ]:
# 1. df_2에 product_sum 컬럼 추가
df_2['product_sum'] = df_2.groupby('product')['product_id'].transform('nunique')

# 2. product_filtered_df에 product_sum 추가
# product_filtered_df에 product 컬럼이 있다고 가정
product_filtered_df = product_filtered_df.merge(
    df_2[['product', 'product_sum']].drop_duplicates(),
    on='product',
    how='left'
)
product_filtered_df

,product,view,cart,purchase,avg_view,avg_cart,avg_purchase,cart_conversion,purchase_conversion,product_sum
0,videocards,97141,12683,6888,3.338,0.436,0.237,0.131,0.543,539
1,printer,37479,3183,2557,1.897,0.161,0.129,0.085,0.803,2291
2,acoustic,24776,1140,848,1.839,0.085,0.063,0.046,0.744,1559
3,motherboard,23221,2113,1266,2.299,0.209,0.125,0.091,0.599,399
4,cpu,21314,2241,1213,2.148,0.226,0.122,0.105,0.541,302
...,...,...,...,...,...,...,...,...,...,...
64,toster,204,3,2,1.427,0.021,0.014,0.015,0.667,52
65,vr,180,9,7,1.417,0.071,0.055,0.050,0.778,3
66,generator,176,6,2,1.323,0.045,0.015,0.034,0.333,11
67,heater,106,6,4,1.359,0.077,0.051,0.057,0.667,35


In [161]:
# product_filtered_df.to_csv('/content/drive/My Drive/product_detail.csv', encoding='utf-8-sig', index=False)

In [157]:
# 상위 10개 구매 전환율 품목
cart_conversion_df = product_filtered_df.sort_values(by='cart_conversion', ascending=False)
cart_conversion_df_10 = cart_conversion_df.head(10)

fig = px.bar(cart_conversion_df_10,
             x='product',
             y='cart_conversion',
             title='Top 10 Cart Conversion by Product',
             labels={'cart_conversion': 'Cart Conversion Rate', 'product': 'Product'},
             color='cart_conversion',
             color_continuous_scale=px.colors.sequential.Viridis,
             text = 'product_sum')

fig.update_traces(textfont_size=16)

fig.show()

In [160]:
# 하위 10개 구매 전환율 품목
cart_conversion_df = product_filtered_df.sort_values(by='cart_conversion', ascending=False)
cart_conversion_df_10 = cart_conversion_df.tail(10)

fig = px.bar(cart_conversion_df_10,
             x='product',
             y='cart_conversion',
             title='Bottom 10 Conversion by Product',
             labels={'cart_conversion': 'Cart Conversion Rate', 'product': 'Product'},
             color='cart_conversion',
             color_continuous_scale=px.colors.sequential.Viridis,
             text = 'product_sum')

fig.update_traces(textfont_size=16)

fig.show()

In [159]:
# purchase_conversion을 기준으로 내림차순 정렬
purchase_conversion_df = product_filtered_df.sort_values(by='purchase_conversion', ascending=False)

# 상위 10개 제품 선택
purchase_conversion_df_10 = purchase_conversion_df.head(10)

# Plotly를 사용하여 바 차트 생성
fig = px.bar(purchase_conversion_df_10,
             x='product',
             y='purchase_conversion',
             title='Top 10 Purchase Conversion by Product',
             labels={'purchase_conversion': 'Purchase Conversion Rate', 'product': 'Product'},
             color='purchase_conversion',
             color_continuous_scale=px.colors.sequential.Viridis,
             text = 'product_sum')

fig.update_traces(textfont_size=16)

# 그래프 표시
fig.show()

In [ ]:
# category가 'electronics'이고 event_type이 'view'인 데이터 필터링
electronics_view_data = df[(df['category'] == 'electronics') & (df['event_type'] == 'view')]

# 브랜드별 view 횟수 집계
brand_views = electronics_view_data.groupby('brand').size().reset_index(name='total_views')

# 내림차순 정렬
brand_views = brand_views.sort_values(by='total_views', ascending=False)

# 결과 출력
print(brand_views)

         brand  total_views
232    samsung        15293
239     sirius         8682
26         bbk         4903
288     xiaomi         4613
150         lg         4327
..         ...          ...
132      jison            1
89      g-case            1
130      jekod            1
24   batterika            1
222     remaks            1

[296 rows x 2 columns]


In [ ]:
# category가 'electronics'이고 event_type이 'view'인 데이터 필터링
electronics_view_data = df[(df['category'] == 'electronics') & (df['event_type'] == 'view')]

# 브랜드별 view 횟수 집계
brand_views = electronics_view_data.groupby('brand').size().reset_index(name='total_views')

# 내림차순 정렬
brand_views = brand_views.sort_values(by='total_views', ascending=False)

# 결과 출력
print(brand_views)

         brand  total_views
232    samsung        15293
239     sirius         8682
26         bbk         4903
288     xiaomi         4613
150         lg         4327
..         ...          ...
132      jison            1
89      g-case            1
130      jekod            1
24   batterika            1
222     remaks            1

[296 rows x 2 columns]


In [ ]:
# 브랜드를 총 뷰 수에 따라 내림차순으로 정렬
sorted_brands = brand_views.sort_values(by='total_views', ascending=False)

# 상위 30개 브랜드 선택
top_brands = sorted_brands.head(50)

# 나머지 브랜드의 총 뷰 수 합산
others_views = sorted_brands.iloc[50:]['total_views'].sum()

# "others" 데이터프레임 생성
others_df = pd.DataFrame({'brand': ['others'], 'total_views': [others_views]})

# 상위 30개 브랜드와 "others" 데이터프레임 합치기
final_data = pd.concat([top_brands, others_df], ignore_index=True)

# Plotly를 사용한 시각화
fig = px.bar(final_data, x='brand', y='total_views',
             title='브랜드별 총 뷰 수',
             labels={'total_views': '총 뷰 수', 'brand': '브랜드'},
             color='total_views',
             text='total_views')

# 텍스트 레이블 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# 레이아웃 설정
fig.update_layout(yaxis_title='Total_View', xaxis_title='Brand')

# 그래프 출력
fig.show()

In [ ]:
# 'view' 이벤트만 필터링
view_events = df[df['event_type'] == 'view']

# 브랜드별 조회 수 계산
view_counts = view_events['brand'].value_counts().reset_index()
view_counts.columns = ['brand', 'views']

# 100회 미만으로 조회된 브랜드 필터링
low_view_brands = view_counts[view_counts['views'] < 100]

# 100회 미만 조회된 브랜드 수
num_low_view_brands = low_view_brands.shape[0]

# 시각화
fig = px.bar(low_view_brands, x='brand', y='views',
             title=f'조회 수가 100회 미만인 브랜드 수: {num_low_view_brands}',
             labels={'views': '조회 수', 'brand': '브랜드'},
             color='views',
             text='views')

# 텍스트 레이블 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# 레이아웃 설정
fig.update_layout(yaxis_title='조회 수', xaxis_title='브랜드', xaxis_tickangle=-45)

# 그래프 출력
fig.show()

# PRODUCT TOP 10

In [ ]:
# product 컬럼의 값별로 행 갯수를 세고, 가장 많은 10개를 추출
top_products = df['product'].value_counts().head(10).index

# top_products에 포함된 product만 남기고 나머지는 제거
top10_df = df[df['product'].isin(top_products)]

top10_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340882 entries, 3 to 885125
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     340882 non-null  object 
 1   event_type     340882 non-null  object 
 2   product_id     340882 non-null  int64  
 3   category_code  340882 non-null  object 
 4   category       340882 non-null  object 
 5   product        340882 non-null  object 
 6   brand          340882 non-null  object 
 7   price          340882 non-null  float64
 8   user_id        340882 non-null  int64  
 9   user_session   340882 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 28.6+ MB


In [ ]:
top10_df['event_time'] = pd.to_datetime(df['event_time'])

# user_id 기준 오름차순, event_time 기준 내림차순 정렬
top10_df = df.sort_values(by=['user_id', 'event_time'], ascending=[True, True])
top10_df.head()

,event_time,event_type,product_id,category_code,category,product,brand,price,user_id,user_session
174670,2020-10-29 11:28:35 UTC,view,4101974,electronics.clocks,electronics,clocks,honor,76.48,1515915625353226922,7qejzWzHlR
242211,2020-11-09 08:52:51 UTC,view,124883,electronics.audio.acoustic,electronics,acoustic,logitech,23.90,1515915625353230683,dn9FkZ11dA
242297,2020-11-09 09:04:34 UTC,view,760044,electronics.audio.acoustic,electronics,acoustic,dell,48.73,1515915625353230683,dn9FkZ11dA
242335,2020-11-09 09:08:53 UTC,view,125325,electronics.audio.acoustic,electronics,acoustic,logitech,23.90,1515915625353230683,dn9FkZ11dA
242344,2020-11-09 09:10:59 UTC,view,461533,electronics.audio.acoustic,electronics,acoustic,smartbuy,12.94,1515915625353230683,dn9FkZ11dA


In [ ]:
import plotly.express as px

# event_type별로 카운트
funnel_data = top10_df['event_type'].value_counts().reset_index()
funnel_data.columns = ['event_type', 'count']

# 퍼널 그래프 그리기
fig = px.funnel(funnel_data, x='count', y='event_type', title='이벤트 타입별 퍼널 그래프')

# 그래프 출력
fig.show()

In [ ]:
cd /content/drive/MyDrive/폴더명